# Json Parsing And Processing

In [2]:
import json
import os
os.makedirs("data/json_files",exist_ok=True)

## Create Data

In [3]:
# Sample nested JSON data
json_data = {
    "company": "TechCorp",
    "employees": [
        {
            "id": 1,
            "name": "John Doe",
            "role": "Software Engineer",
            "skills": ["Python", "JavaScript", "React"],
            "projects": [
                {"name": "RAG System", "status": "In Progress"},
                {"name": "Data Pipeline", "status": "Completed"}
            ]
        },
        {
            "id": 2,
            "name": "Jane Smith",
            "role": "Data Scientist",
            "skills": ["Python", "Machine Learning", "SQL"],
            "projects": [
                {"name": "ML Model", "status": "In Progress"},
                {"name": "Analytics Dashboard", "status": "Planning"}
            ]
        }
    ],
    "departments": {
        "engineering": {
            "head": "Mike Johnson",
            "budget": 1000000,
            "team_size": 25
        },
        "data_science": {
            "head": "Sarah Williams",
            "budget": 750000,
            "team_size": 15
        }
    }
}

In [4]:
json_data

{'company': 'TechCorp',
 'employees': [{'id': 1,
   'name': 'John Doe',
   'role': 'Software Engineer',
   'skills': ['Python', 'JavaScript', 'React'],
   'projects': [{'name': 'RAG System', 'status': 'In Progress'},
    {'name': 'Data Pipeline', 'status': 'Completed'}]},
  {'id': 2,
   'name': 'Jane Smith',
   'role': 'Data Scientist',
   'skills': ['Python', 'Machine Learning', 'SQL'],
   'projects': [{'name': 'ML Model', 'status': 'In Progress'},
    {'name': 'Analytics Dashboard', 'status': 'Planning'}]}],
 'departments': {'engineering': {'head': 'Mike Johnson',
   'budget': 1000000,
   'team_size': 25},
  'data_science': {'head': 'Sarah Williams',
   'budget': 750000,
   'team_size': 15}}}

In [5]:
with open('data/json_files/company_data.json', 'w') as f:
    json.dump(json_data, f, indent=2)

In [6]:
# Save JSON Lines format
jsonl_data = [
    {"timestamp": "2024-01-01", "event": "user_login", "user_id": 123},
    {"timestamp": "2024-01-01", "event": "page_view", "user_id": 123, "page": "/home"},
    {"timestamp": "2024-01-01", "event": "purchase", "user_id": 123, "amount": 99.99}
]

with open('data/json_files/events.jsonl', 'w') as f:
    for item in jsonl_data:
        f.write(json.dumps(item) + '\n')

## Json Processing Stratergies

### Method 1: Using JSONLoader with jq_schema

In [7]:
from langchain_community.document_loaders import JSONLoader
import json

# ----------------------------------------------------------
# Method 1: Using JSONLoader with jq_schema
# ----------------------------------------------------------
# Purpose:
#   To load and extract *specific fields or nested objects* from
#   a JSON file using a jq-style query.
#   This method is perfect for semi-structured JSON data like
#   employee records, products, or API responses.
# ----------------------------------------------------------

print("1️⃣ JSONLoader - Extract specific fields")

# Create a JSONLoader instance
employee_loader = JSONLoader(
    file_path='data/json_files/company_data.json',  # Path to your JSON file
    jq_schema='.employees[]',                       # jq query → extract each item inside the "employees" array
    text_content=False                              # False = keep full JSON objects instead of plain text
)

# Load the data
#    - Each extracted JSON object becomes one `Document` in LangChain
#    - The loader automatically attaches metadata like file path, line number, etc.
employee_docs = employee_loader.load()

# Print how many employee documents were extracted
print(f"Loaded {len(employee_docs)} employee documents")

# Show a short preview of the first employee document
print(f"First employee: {employee_docs[0].page_content[:200]}...")

# (Optional) Print all employee docs — useful for inspection
print(employee_docs)


1️⃣ JSONLoader - Extract specific fields
Loaded 2 employee documents
First employee: {"id": 1, "name": "John Doe", "role": "Software Engineer", "skills": ["Python", "JavaScript", "React"], "projects": [{"name": "RAG System", "status": "In Progress"}, {"name": "Data Pipeline", "status"...
[Document(metadata={'source': 'C:\\Users\\Panduka Bandara\\Desktop\\RAG-A-Z\\RAG-hello-world\\0-Data-Ingest-Parsing\\data\\json_files\\company_data.json', 'seq_num': 1}, page_content='{"id": 1, "name": "John Doe", "role": "Software Engineer", "skills": ["Python", "JavaScript", "React"], "projects": [{"name": "RAG System", "status": "In Progress"}, {"name": "Data Pipeline", "status": "Completed"}]}'), Document(metadata={'source': 'C:\\Users\\Panduka Bandara\\Desktop\\RAG-A-Z\\RAG-hello-world\\0-Data-Ingest-Parsing\\data\\json_files\\company_data.json', 'seq_num': 2}, page_content='{"id": 2, "name": "Jane Smith", "role": "Data Scientist", "skills": ["Python", "Machine Learning", "SQL"], "projects": [{"na

### Method 2: Custom JSON processing for complex structures

In [8]:
# Method 2: Custom JSON processing for complex structures
from typing import List
from langchain_core.documents import Document
import json

print("\n2️⃣ Custom JSON Processing")

def process_json_intelligently(filepath: str) -> List[Document]:
    """
    Process a JSON file intelligently:
      - Flattens nested structures (like employee projects)
      - Preserves context in readable format
      - Adds rich metadata for retrieval
    """

    # Open and load JSON file
    with open(filepath, 'r') as f:
        data = json.load(f)

    # Initialize a list to hold Document objects
    documents = []

    # Iterate over 'employees' array in JSON
    for emp in data.get('employees', []):
        # Build a readable, structured content string
        content = f"""Employee Profile:
        Name: {emp['name']}
        Role: {emp['role']}
        Skills: {', '.join(emp['skills'])}

        Projects:"""
        
        # Add each project under the employee
        for proj in emp.get('projects', []):
            content += f"\n- {proj['name']} (Status: {proj['status']})"
        
        # Create a LangChain Document for this employee
        doc = Document(
            page_content=content,  # The human-readable content
            metadata={             # Rich metadata for filtering/search
                'source': filepath,
                'data_type': 'employee_profile',
                'employee_id': emp['id'],
                'employee_name': emp['name'],
                'role': emp['role']
            }
        )

        # Append the Document to the list
        documents.append(doc)

    # Return all processed Documents
    return documents



2️⃣ Custom JSON Processing


In [9]:
process_json_intelligently("data/json_files/company_data.json")

[Document(metadata={'source': 'data/json_files/company_data.json', 'data_type': 'employee_profile', 'employee_id': 1, 'employee_name': 'John Doe', 'role': 'Software Engineer'}, page_content='Employee Profile:\n        Name: John Doe\n        Role: Software Engineer\n        Skills: Python, JavaScript, React\n\n        Projects:\n- RAG System (Status: In Progress)\n- Data Pipeline (Status: Completed)'),
 Document(metadata={'source': 'data/json_files/company_data.json', 'data_type': 'employee_profile', 'employee_id': 2, 'employee_name': 'Jane Smith', 'role': 'Data Scientist'}, page_content='Employee Profile:\n        Name: Jane Smith\n        Role: Data Scientist\n        Skills: Python, Machine Learning, SQL\n\n        Projects:\n- ML Model (Status: In Progress)\n- Analytics Dashboard (Status: Planning)')]